# AF detection using RRI mean

We first extract the x coordinate of each r_peak in a single normal sinus rythme ECG signal. To sea the plot of this ECG signal, please refer to the notebook [extraction.ipynb](ecg-features/notebooks/extraction.ipynb) (index 0).

In [1]:
import numpy as np
from concrete import fhe

In [ ]:
def analysis_space_between_r_peaks(rpeaks_x):

    # Print the rpeaks
    print("Rpeaks:", rpeaks_x)

    # Compute the horizontal distance between each pair of points
    distances = []
    for i in range(len(rpeaks_x)-1):
        x1 = rpeaks_x[i]
        x2 = rpeaks_x[i+1]
        distance = abs(x2 - x1)
        distances.append(distance)

        if i % 6 == 0:print("")

        print(i, "->", i+1, ":", distance, end=" || ")
    print("")

    # Compute the mean of the distances
    mean_distance = sum(distances) / len(distances)
    print("\nMean distance:", mean_distance)

    # Compute the median of the distances
    sorted_distances = sorted(distances)
    if len(sorted_distances) % 2 == 0:
        median_distance = (sorted_distances[len(sorted_distances)//2-1] + sorted_distances[len(sorted_distances)//2]) / 2
    else:
        median_distance = sorted_distances[len(sorted_distances)//2]
    print("Median distance:", median_distance, end="\n\n")

    # If there is a distance that is more than 3% different from the mean, display it
    threshold = 0.03

    counter_of_outliers = 0

    outliers_table = []

    for i in range(len(distances)):
        distance = distances[i]
        if distance < mean_distance * (1 - threshold) or distance > mean_distance * (1 + threshold):
            print("Distance", i, "to", i+1, "is", distance, "which is more than", threshold*100, "% different from the mean")
            counter_of_outliers += 1
            outliers_table.append(1)
        else:
            outliers_table.append(0)
    # Print the number of outliers with warn message about a possible Atrial fibrillation
    if counter_of_outliers > 0:
        print("\nWARNING: There are", counter_of_outliers, "outliers. This may be a sign of Atrial fibrillation")

    # Print the outliers table
    print("\nOutliers table:", outliers_table)

In [3]:
rpeaks_x = np.array(np.load('ecg-features/extracted_data/coord_x_r_peaks_A00021.npy'))

analysis_space_between_r_peaks(rpeaks_x)

Rpeaks: [ 151  427  706  993 1282 1562 1849 2138 2419 2695 2981 3258 3528 3812
 4088 4363 4651 4940 5215 5493 5779 6065 6341 6624 6911 7186 7473 7764
 8040 8329 8615]

0 -> 1 : 276 || 1 -> 2 : 279 || 2 -> 3 : 287 || 3 -> 4 : 289 || 4 -> 5 : 280 || 5 -> 6 : 287 || 
6 -> 7 : 289 || 7 -> 8 : 281 || 8 -> 9 : 276 || 9 -> 10 : 286 || 10 -> 11 : 277 || 11 -> 12 : 270 || 
12 -> 13 : 284 || 13 -> 14 : 276 || 14 -> 15 : 275 || 15 -> 16 : 288 || 16 -> 17 : 289 || 17 -> 18 : 275 || 
18 -> 19 : 278 || 19 -> 20 : 286 || 20 -> 21 : 286 || 21 -> 22 : 276 || 22 -> 23 : 283 || 23 -> 24 : 287 || 
24 -> 25 : 275 || 25 -> 26 : 287 || 26 -> 27 : 291 || 27 -> 28 : 276 || 28 -> 29 : 289 || 29 -> 30 : 286 || 

Mean distance: 282.1333333333333
Median distance: 283.5

Distance 11 to 12 is 270 which is more than 3.0 % different from the mean
Distance 26 to 27 is 291 which is more than 3.0 % different from the mean


Outliers table: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

## Comparing with FHE

In [ ]:
threshold = 0.03
modified_threshold = int(threshold*100)

@fhe.compiler({"r_peaks": "encrypted"})
def analysis_space_between_r_peaks_fhe(r_peaks):

    len_rri = r_peaks.size
    
    for i in range(len_rri-1):
        r_peaks[i] = r_peaks[i+1]-r_peaks[i]

    # version that do not use TLU (and so no float) and is faster
    peaks_mean = np.floor_divide(sum(r_peaks[:-1]) , (len_rri-1))-modified_threshold

    # version that use a TLU and is slower (~32min of execution)
    # peaks_mean = (sum(r_peaks[:-1]) / (len_rri-1)).astype(np.int64)-modified_threshold

    for i in range(len_rri):
        r_peaks[i] = r_peaks[i] < peaks_mean
    
    return r_peaks

In [ ]:
inputset = [np.random.randint(9000, size=31), np.random.randint(9000, size=31),np.random.randint(9000, size=31)]
circuit = analysis_space_between_r_peaks_fhe.compile(inputset)

In [ ]:
homomorphic_evaluation = circuit.encrypt_run_decrypt(rpeaks_x)
print(homomorphic_evaluation)

## Conclusion
we notive that the results are very close. Our small homemade algorithms work correctly. That said, we note that the FHE version did not raise any warning regarding the distance between the peaks 26 and 27. This is due to the fact that the homemade algorithm implemented with FHE does not take into account cases where the distance between two peaks is "too large". This is why it only raises the warning for the distance between the peaks 11 and 12.